<a href="https://colab.research.google.com/github/Grace-Gitome/Data-Science-BootCamp-2023/blob/main/sentimental_analysis_of_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [2]:
#Uploading file from local disk
from google.colab import files
uploaded = files.upload()

Saving training.1600000.processed.noemoticon.csv to training.1600000.processed.noemoticon.csv


# Data Collection and Preprocessing

In [6]:
#Store the dataset into a dataframe
df =pd.read_csv('training.1600000.processed.noemoticon.csv',encoding='latin-1' )

In [7]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [8]:
#renaming columns
df.columns =['target','id', 'date', 'flag', 'user', 'text']

In [9]:
#view the column names
df.columns.values

array(['target', 'id', 'date', 'flag', 'user', 'text'], dtype=object)

In [10]:
df.tail()

,target,id,date,flag,user,text
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599998,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

### Exploratory Data Analysis

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1599999 non-null  int64 
 1   id      1599999 non-null  int64 
 2   date    1599999 non-null  object
 3   flag    1599999 non-null  object
 4   user    1599999 non-null  object
 5   text    1599999 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [12]:
df.shape

(1599999, 6)

There are 1599999 rows and 6 columns in the datset

In [13]:
df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

There are no nulls

In [14]:
duplicate_rows = df[df.duplicated()]
duplicate_rows

,target,id,date,flag,user,text


There are no duplicates

In [15]:
#Check number of target variables 
df['target'].nunique()

2

The tweets have been annotated (0 = negative, 4 = positive) and they can be used to detect sentiment .

### Text Pre-processing

Text data is unstructured in nature.

#### Sentiment classification

1. **Text** - actual review comment. <br>
2. **Target** -the tweets have been annotated (0 = negative, 4 = positive) and they can be used to detect sentiment<br>

Since we are only concerned with columns of unstructured textual tweets and sentiments, the remaining columns are dropped.

In [16]:
#Dropping the unnecessary columns from the dataset

df1 = df.drop(['id', 'date', 'flag', 'user'],axis = 1)
df1.head(7)

,target,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew
5,0,Need a hug
6,0,@LOLTrish hey long time no see! Yes.. Rains a...


### Lowercase all the tweets

The first step is transforming the tweets into lowercase to maintain a consistent flow during the NLP tasks and text mining.<br> 
For example 'Nation' and 'nation' will be treated as two different words in any sentence, and hence, we need to make all the words lowercase in the tweets to avoid duplication while creating our words dictionary.

In [17]:
df1['text'] = df1 ['text'].str.lower()
df1.head()

,target,text
0,0,is upset that he can't update his facebook by ...
1,0,@kenichan i dived many times for the ball. man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@kwesidei not the whole crew


### Remove Hyper-Links
They are very common in tweets and don't add any additional information as per our problem statement of sentiment analysis.

In [18]:
#Removing hyper-links
df1['text'] = df1['text'].str.replace(r"http:\S+","",regex = True)
df1.head()

,target,text
0,0,is upset that he can't update his facebook by ...
1,0,@kenichan i dived many times for the ball. man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@kwesidei not the whole crew


### Remove punctuations
For most NLP problems, punctuations do not provide additional language information and are generally removed.<br>
Similarly, punctuation symbols are not crucial for sentiment analysis. they are redundant are the removal of punctuation before text modelling is highly recommended.

In [19]:
#Removing punctuations
df1['text'] = df1['text'].str.replace('[^A-Za-z0-9]+', ' ', regex =True)
df1.head()

,target,text
0,0,is upset that he can t update his facebook by ...
1,0,kenichan i dived many times for the ball mana...
2,0,my whole body feels itchy and like its on fire
3,0,nationwideclass no it s not behaving at all i...
4,0,kwesidei not the whole crew


### Remove Stopwords
Stopwords are English words that do not add much meaning to a sentence. They are removed as they do not add value to the analysis.

NLTK library consists of a list of words that are considered stopwords for the English language. Some of them are : [i, me, my, myself, we, our, ours]

In [28]:
#requirement to remove stopwords

#importing nlp library
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:

#NLTK library provides a set of stopwords for English
from nltk.corpus import stopwords 

#Stopwords present in the library
stopwords = set(stopwords.words('english'))


#Removing stopwords
df1['text'] = df1['text'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stopwords))
df1.head()

,target,text
0,0,upset update facebook texting might cry result...
1,0,kenichan dived many times ball managed save 50...
2,0,whole body feels itchy like fire
3,0,nationwideclass behaving mad see
4,0,kwesidei whole crew


### Tokenization

This refers to splitting up a larger body of text into smaller lines (paragraphs) and words. These broken pieces are called tokens (either word token or sentence tokens)

Below is  random sample code to illustrate tokenization

In [26]:
#Requirement for tokenization
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
#importing nlp library
import nltk

string = "What is your favourite food joint?"

tokens = nltk.word_tokenize(string)

print(tokens)

['What', 'is', 'your', 'favourite', 'food', 'joint', '?']


In [31]:
df2 = df1.copy()

In [33]:
#defining function for tokenization
#Tokenization of tweets using the TweetTokenizer
from nltk.tokenize import TweetTokenizer
def tokenize(text):
  tknzr = TweetTokenizer(reduce_len=True)
  return tknzr.tokenize(text)

#applying function to the column
df2['tokenized_text']=[tokenize(sentence) for sentence in df2['text']]
df2.head()


,target,text,tokenized_text
0,0,upset update facebook texting might cry result...,"[upset, update, facebook, texting, might, cry,..."
1,0,kenichan dived many times ball managed save 50...,"[kenichan, dived, many, times, ball, managed, ..."
2,0,whole body feels itchy like fire,"[whole, body, feels, itchy, like, fire]"
3,0,nationwideclass behaving mad see,"[nationwideclass, behaving, mad, see]"
4,0,kwesidei whole crew,"[kwesidei, whole, crew]"


### Stemming and Lemmatization

These are text normalization techniques for NLP.

Definition of terms:<br>
WordNet - a large lexical database of English Words.<br>

Punkt -a tokenizer that devides a text into a list of sentences using an unsuprevised algorith to build a model for abbreviation words and words that start with sentences.

WhitespaceTokenizer()- this method breaks texts into terms whenever it encounters a whitespace character.<br><br>
WordNetLemmatizer() - lemmatize a word on its context usage within the sentence

In [ ]:
#Applying word tokenization and lemmatization to the tweets

lemmatizer =nltk.stem.WordNetLemmatizer()

#A fucntion to perform lemmatization on the text
def lemm (text):
    return [lemmatizer.lemmatize(w) for w in word_tokenizer.tokenize(text)]


#Create column for lemmatized tokens
df1['lemmatized_tokens '] = df1['text'].apply(lemm)
df1.head()



# Model Development

In [ ]:
#Importing necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import (LinearRegression,
                                         Ridge,
                                         Lasso)
from sklearn.metrics import (r2_score,
                             mean_absolute_error,
                             mean_squared_error)